In [5]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_hub as hub
import dataset as d
import random
from tqdm import tqdm

In [6]:
num_words = 250


In [7]:
# grab the universal sentence encoder from tf hub
# embed_use = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")

In [8]:
data = d.load_data(1000000)
train, test = d.split_data(data)
train = d.process_data(train, 10, 100)
x, y = d.shuffle_and_split(train)


999999it [00:26, 37339.25it/s]
100%|██████████| 900000/900000 [00:23<00:00, 39050.23it/s]


In [9]:
tokenizer = d.create_tokenizer(data)
vocab_size = len(tokenizer.word_index) + 1

In [10]:
input_text = keras.Input((), dtype = tf.string, name = 'input_text')
embedding = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder-large/5", input_shape = [], dtype = tf.string, trainable = False)(input_text)
dense = layers.Dense(512 * 2, activation="relu")(embedding)
dense = layers.Dropout(0.3)(dense)
dense = layers.Dense(512 * 4, activation="relu")(dense)
dense = layers.Dropout(0.2)(dense)
dense = layers.Dense(512 * 8, activation="relu")(dense)
dense = layers.Dropout(0.2)(dense)
dense = layers.Dense(512 * 8, activation="relu")(dense)
dense = layers.Dropout(0.1)(dense)
dense = layers.Dense(512 * 8, activation="relu")(dense)
dense = layers.Dropout(0.1)(dense)
dense = layers.Dense(512 * 8, activation="relu")(dense)
dense = layers.Dropout(0.1)(dense)
dense = layers.Dense(512 * 8, activation="relu")(dense)
dense = layers.Dropout(0.1)(dense)
dense = layers.Dense(512 * 4, activation="relu")(dense)
dense = layers.Dropout(0.05)(dense)
dense = layers.Dense(512 * 2, activation="relu")(dense)
dense = layers.Dense(512 * 1, activation="relu")(dense)
output = layers.Dense(num_words, activation="softmax")(dense)

In [11]:
model = keras.Model(inputs=input_text, outputs=output)

In [12]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_text (InputLayer)      [(None,)]                 0         
_________________________________________________________________
keras_layer (KerasLayer)     (None, 512)               147354880 
_________________________________________________________________
dense (Dense)                (None, 1024)              525312    
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 2048)              2099200   
_________________________________________________________________
dropout_1 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 4096)              839270

In [ ]:
# Y = []
# X = []

# one_hot = np.eye(num_words, dtype='int8')


# for _x, _y in zip(x, y):
#     if _y in tokenizer.word_index:
#         if tokenizer.word_index[_y] < num_words:
      
#             Y.append(one_hot[tokenizer.word_index[_y]])
#             X.append(_x)
        
        
# Y = np.array(Y)
# X = np.array(X)

In [13]:
_Y = []
_X = []

one_hot = np.eye(num_words, dtype='int8')

word_count = {}

for _x, _y in zip(x, y):
    if _y in tokenizer.word_index:
        if tokenizer.word_index[_y] < num_words:
            if _y not in word_count:
                word_count[_y] = 1
            else:
                word_count[_y] += 1
                
      
            _Y.append(_y)
            _X.append(_x)
        


In [15]:
min_words = word_count[tokenizer.index_word[num_words - 10]]
X = []
Y = []

word_count = {}
for _x, _y in zip(_X, _Y):
    if _y not in word_count:
        word_count[_y] = min_words
    if word_count[_y] != 0:
        Y.append(one_hot[tokenizer.word_index[_y]])
        X.append(_x)
        
        word_count[_y] -= 1

In [20]:
X = np.array(X)
Y = np.array(Y)

In [21]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [24]:
model.fit(x=X, y=Y, batch_size=256, epochs=8, validation_split=0.02)

Epoch 1/8
  19/1406 [..............................] - ETA: 3:35 - loss: 4.6009 - accuracy: 0.1055

KeyboardInterrupt: 

In [ ]:
tokenizer.word_index['the']

In [51]:
a = model.predict(['communism'])

In [52]:
tokenizer.index_word[tf.argmax(tf.reshape(a, (-1))).numpy()]

'party'

In [100]:
import pickle
model.save("model_data/model.h5")


In [101]:
with open('model_data/corpus.pickle', 'wb') as corpus:
    pickle.dump([tokenizer.word_index, tokenizer.index_word], corpus)